<a href="https://colab.research.google.com/github/dcpatton/Image-Classification/blob/master/cifar10_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective



> Just a simple example of image classification with a CNN in TensorFlow



In [67]:
import tensorflow as tf
import random

seed = 52
tf.random.set_seed(seed)
random.seed(seed)

tf.__version__

'2.3.0'

# Get the data

In [68]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [69]:
print(train_images.shape)
print(test_images.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


# Model

In [70]:
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomRotation
from tensorflow.keras import Input, Model

inp = Input(shape=(32, 32, 3), name='inp')
# x = RandomRotation(0.1, seed=seed)(inp)
x = Rescaling(1./255)(x)
x = Conv2D(64, 5, activation='relu')(inp)
x = Dropout(0.25)(x)
x = MaxPool2D()(x)
x = Conv2D(128, 3, activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPool2D()(x)
x = Conv2D(256, 3, activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
out = Dense(10, activation='softmax', name='out')(x)

model = Model(inputs=[inp], outputs=[out])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics=['acc'])

model.summary()

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 28, 28, 64)        4864      
_________________________________________________________________
dropout_3 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 12, 128)       0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 6, 6, 128)       

# Training

In [71]:
filepath = 'model.h5'
mc = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, 
                                        save_weights_only=True)
es = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)

history = model.fit(train_images, train_labels, epochs=200, 
                    validation_data=(test_images, test_labels), 
                    callbacks=[mc, es], verbose=1, batch_size=16)

Epoch 1/200
1563/1563 [==============================] - 6s 4ms/step - loss: 2.0064 - acc: 0.3112 - val_loss: 1.7485 - val_acc: 0.4136
Epoch 2/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5927 - acc: 0.4244 - val_loss: 1.5934 - val_acc: 0.4580
Epoch 3/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4952 - acc: 0.4677 - val_loss: 1.5192 - val_acc: 0.4843
Epoch 4/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4504 - acc: 0.4860 - val_loss: 1.4589 - val_acc: 0.5235
Epoch 5/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4162 - acc: 0.5016 - val_loss: 1.4758 - val_acc: 0.5150
Epoch 6/200
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3693 - acc: 0.5194 - val_loss: 1.3711 - val_acc: 0.5454
Epoch 7/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.3353 - acc: 0.5369 - val_loss: 1.4569 - val_acc: 0.4982
Epoch 8/200
1563/1563 [==============================] 

# Evaluation

In [72]:
model.load_weights(filepath)
metrics = model.evaluate(test_images, test_labels, verbose=0)
print('Loss = ' + str(metrics[0]))
print('Accuracy = ' + str(metrics[1]))

Loss = 1.0439012050628662
Accuracy = 0.6722999811172485


Loss = 0.7913779616355896

Accuracy = 0.7289000153541565